In [ ]:
# Author: Arthur BARREAU
# Date: 2025-07-15
# R version: 4.3.3
# Description: Downloads NSIDC daily sea ice data, classifies presence, aggregates 
#              by subarea, and exports a masked raster map with labels and borders.
# Source data: https://noaadata.apps.nsidc.org/NOAA/G02135/south/daily/geotiff/

# ==============================================================================
# 0. PACKAGE & LIBRARY INSTALLATION ----
# ==============================================================================
options(timeout = 3600)

system("conda install -c conda-forge libgdal-hdf5")
system("conda install -y r-terra=1.8_42 r-sf=1.0_20 r-gridExtra=2.3 r-ncdf4 r-fields")

library(terra)
library(sf)
library(gridExtra)
library(dplyr)
library(ggplot2)
library(ncdf4)
library(jsonlite)
library(fields)
library(stringr)

# ==============================================================================
# 1. PARAMETERS AND INPUT LOADING ----
# ==============================================================================
json_data <- fromJSON("galaxy_inputs/galaxy_inputs.json")

file_copernicus <- json_data$CopernicusXDataXUser$path 
subarea         <- json_data$subarea       
area_list       <- trimws(unlist(strsplit(subarea, ",")))
multiplier      <- json_data$multiplier
index_max       <- json_data$indexXmax 
legend_pos      <- json_data$legendXposition
month_mode      <- json_data$MonthXmode
nb_row          <- json_data$nbXrow
asd_show        <- json_data$ASD
title           <- json_data$titile
bbox            <- json_data$bbox
png             <- json_data$png
ext             <- ifelse(png, ".png", ".pdf")
# ==============================================================================
# 2. LOAD NETCDF & GEBCO DATA ----
# ==============================================================================

nc <- nc_open(file_copernicus)
time <- ncvar_get(nc, "time")
lat <- ncvar_get(nc, "latitude")
lon <- ncvar_get(nc, "longitude")
chl_var <- ncvar_get(nc, "CHL")
dates <- as.POSIXct(time, origin = "1970-01-01", tz = "UTC")

In [ ]:
# ==============================================================================
# 3. PREPARE CHL DATA ACCORDING TO MODE ----
# ==============================================================================

if (month_mode == "single_month") {
  chl_data <- chl_var[, ncol(chl_var):1]  # reverse columns
  filename <- paste0("outputs/collection/Fig9_", format(dates[1], "%Y-%m"), ext)

} else if (month_mode == "monthly_mean") {
  chl_mean <- apply(chl_var, c(1, 2), mean)
  chl_data <- chl_mean[, ncol(chl_mean):1]
  start_str <- format(dates[1], "%Y-%m")
  end_str <- format(dates[length(dates)], "%Y-%m")
  filename <- paste0("outputs/collection/Fig9_", start_str, "_", end_str, ext)
}

# Create raster
if (month_mode %in% c("single_month", "monthly_mean")) {
chl_raster <- rast(nrows = length(lat), ncols = length(lon),
                   xmin = min(lon), xmax = max(lon),
                   ymin = min(lat), ymax = max(lat),
                   crs = "EPSG:4326")
values(chl_raster) <- as.vector(chl_data)
chl_projected <- project(chl_raster, "EPSG:6932")
}
# Load GEBCO raster
gebco_url <- "https://gis.ccamlr.org/geoserver/www/GEBCO2024_5000.tif"
download.file(gebco_url, "GEBCO2024_5000.tif", mode = "wb")
gebco_raster <- rast("GEBCO2024_5000.tif")

# ==============================================================================
# 4. HANDLE ZONE (ASD or NSWE) ----
# ==============================================================================

letters_only <- str_extract(area_list, "[A-Za-z]+")
numbers_only <- str_extract(area_list, "-?\\d+")

if (is.na(letters_only[1])) {
  # --- ASD ---
  dir.create("asd", showWarnings = FALSE)
  suffixes <- c("shp", "shx", "dbf", "prj")
  asd_urls <- paste0("https://raw.githubusercontent.com/ccamlr/data/refs/tags/v0.5.0/geographical_data/asd/asd-shapefile-EPSG6932.", suffixes)
  lapply(seq_along(asd_urls), function(i) download.file(asd_urls[i], file.path("asd", basename(asd_urls[i])), mode = "wb"))

  ASD_shapes <- st_read("asd/asd-shapefile-EPSG6932.shp", quiet = TRUE)
  selected_areas <- ASD_shapes %>% filter(GAR_Short_ %in% area_list)
  area_vector <- vect(selected_areas)
  extent_obj <- ext(area_vector)
 

         
  if (month_mode %in% c("single_month", "monthly_mean")) {
      if (bbox) {
        subareas_bbox <- vect(st_as_sfc(st_bbox(selected_areas)))
        chl_projected <- mask(crop(chl_projected, subareas_bbox), subareas_bbox)
      } else {
       chl_projected <- mask(crop(chl_projected, area_vector), area_vector)
      }
  }
  if (bbox) {
    subareas_bbox <- vect(st_as_sfc(st_bbox(selected_areas)))
    gebco_masked <- mask(crop(gebco_raster, subareas_bbox), subareas_bbox)
  } else {
     gebco_masked <- mask(crop(gebco_raster, area_vector), area_vector)
  }

} else {
  # --- NSWE ---
  bounds <- list(
    N = as.numeric(numbers_only[letters_only == "N"]),
    S = as.numeric(numbers_only[letters_only == "S"]),
    W = as.numeric(numbers_only[letters_only == "W"]),
    E = as.numeric(numbers_only[letters_only == "E"])
  )
  zone_extent <- ext(bounds$W, bounds$E, bounds$S, bounds$N)
  projected_zone <- project(rast(ext = zone_extent, crs = "EPSG:4326"), "EPSG:6932")
  area_vector <- NULL
  extent_obj <- ext(projected_zone)
  gebco_masked <- mask(crop(gebco_raster, extent_obj), extent_obj)
  if (month_mode %in% c("single_month", "monthly_mean")) {
        chl_projected <- mask(crop(chl_projected, extent_obj), extent_obj)
  }
  if (asd_show) {
      dir.create("asd", showWarnings = FALSE)
      suffixes <- c("shp", "shx", "dbf", "prj")
      asd_urls <- paste0("https://raw.githubusercontent.com/ccamlr/data/refs/tags/v0.5.0/geographical_data/asd/asd-shapefile-EPSG6932.", suffixes)
      lapply(seq_along(asd_urls), function(i) download.file(asd_urls[i], file.path("asd", basename(asd_urls[i])), mode = "wb"))
      ASD <- st_read("asd/asd-shapefile-EPSG6932.shp", quiet = TRUE)
      selected_areas <- ASD_shapes %>% filter(GAR_Short_ %in% c("881","882","883","481","482","483","484","485","486","5841","5842","5843a","5843b","5844a","5844b","5851","5852","586","587"))
      area_vector <- vect(selected_areas)
  }
}

width_img <- abs(extent_obj[2] - extent_obj[1]) / 10000
height_img <- abs(extent_obj[4] - extent_obj[3]) / 10000

# ==============================================================================
# 5. EXPORT STATIC FIGURE (MEAN or SINGLE MONTH) ----
# ==============================================================================

if (month_mode %in% c("single_month", "monthly_mean")) {
  if (png) {
    png(filename, width = width_img * multiplier, height = height_img * multiplier)
  } else {
    pdf(filename)
  }

  # Define palettes
  chl_palette <- colorRampPalette(c(
    "#0286c4", "#1698b7", "#33a1b8", "#4bafad", "#6abbb4",
    "#82cca9", "#94dba1", "#a5e49d", "#b6eba3", "#d6f1ac",
    "#e2f9ac", "#f1feb8", "#fffdb3", "#fff2a4", "#ffe493",
    "#fed48a", "#ffcb6e", "#fcbc65", "#f8b253", "#ff9645",
    "#ff642c", "#f54d29", "#f82619", "#f40616"
  ))(1000)
  chl_palette <- c(chl_palette, "red")
  chl_breaks <- c(seq(0.03, index_max, length.out = 100), Inf)

  plot(gebco_masked, col = c("white", "grey"), breaks = c(-Inf, 0, Inf),
       legend = FALSE, axes = FALSE, box = FALSE)
  plot(chl_projected, col = chl_palette, breaks = chl_breaks,
       legend = FALSE, add = TRUE)

  if (asd_show) {
    plot(area_vector, add = TRUE, lwd = 0.75 * multiplier, border = 'black')
    centroids <- st_centroid(st_geometry(selected_areas))
    text(st_coordinates(centroids), labels = selected_areas$GAR_Long_L,
         col = "black", cex = 1 * multiplier)
  }

  dev.off()
}

In [ ]:
# ==============================================================================
# 6. MULTIPLE PLOTS PER MONTH (GRID OF GGPlots) ----
# ==============================================================================

if (month_mode == "one_plot_per_month") {
  chl_palette <- colorRampPalette(c(
    "#0286c4", "#1698b7", "#33a1b8", "#4bafad", "#6abbb4",
    "#82cca9", "#94dba1", "#a5e49d", "#b6eba3", "#d6f1ac",
    "#e2f9ac", "#f1feb8", "#fffdb3", "#fff2a4", "#ffe493",
    "#fed48a", "#ffcb6e", "#fcbc65", "#f8b253", "#ff9645",
    "#ff642c", "#f54d29", "#f82619", "#f40616"
  ))(1000)
  chl_palette <- c(chl_palette, "red")
  chl_breaks <- c(seq(0.03, index_max, length.out = 100), Inf)

  # === Définir une figure PNG avec dimensions dynamiques ===
  chl_i_sample <- chl_var[, , 1][, ncol(chl_var):1]
  chl_r_sample <- rast(nrows = length(lat), ncols = length(lon),
                       xmin = min(lon), xmax = max(lon),
                       ymin = min(lat), ymax = max(lat),
                       crs = "EPSG:4326")
  values(chl_r_sample) <- as.vector(chl_i_sample)
  chl_proj_sample <- project(chl_r_sample, "EPSG:6932")
  raster_extent <- ext(chl_proj_sample)  # xmin, xmax, ymin, ymax

  w <- abs(raster_extent[2] - raster_extent[1]) / 10000
  h <- abs(raster_extent[4] - raster_extent[3]) / 10000

  nombre_de_figure_par_ligne <- nb_row
  nombre_total_figures <- dim(chl_var)[3]
  nombre_de_lignes <- ceiling(nombre_total_figures / nombre_de_figure_par_ligne)

  start_str <- format(dates[1], "%Y-%m")
  end_str   <- format(dates[length(dates)], "%Y-%m")
  filename  <- paste0("outputs/collection/Fig9_", start_str, "_", end_str, ext)

  if (png) {
    png(filename, width = w * nombre_de_figure_par_ligne * multiplier,
        height = h * nombre_de_lignes * multiplier)
  } else {
    pdf(filename)
  }
  par(mfrow = c(nombre_de_lignes, nombre_de_figure_par_ligne), mar = c(2, 2, 2, 2))


    if (asd_show) {
      dir.create("asd", showWarnings = FALSE)
      suffixes <- c("shp", "shx", "dbf", "prj")
      asd_urls <- paste0("https://raw.githubusercontent.com/ccamlr/data/refs/tags/v0.5.0/geographical_data/asd/asd-shapefile-EPSG6932.", suffixes)
      lapply(seq_along(asd_urls), function(i) download.file(asd_urls[i], file.path("asd", basename(asd_urls[i])), mode = "wb"))
      ASD <- st_read("asd/asd-shapefile-EPSG6932.shp", quiet = TRUE)
      if (is.na(letters_only[1])) {
          selected_areas <- ASD_shapes %>% filter(GAR_Short_ %in% area_list)
          area_vector <- vect(selected_areas)
      } else {
          selected_areas <- ASD_shapes %>% filter(GAR_Short_ %in% c("881","882","883","481","482","483","484","485","486","5841","5842","5843a","5843b","5844a","5844b","5851","5852","586","587"))
          area_vector <- vect(selected_areas)
      }
  }

    for (i in 1:dim(chl_var)[3]) {
        gc()
        chl_i <- chl_var[, , i][, ncol(chl_var):1]
        chl_r <- rast(nrows = length(lat), ncols = length(lon),
                      xmin = min(lon), xmax = max(lon),
                      ymin = min(lat), ymax = max(lat),
                      crs = "EPSG:4326")
        values(chl_r) <- as.vector(chl_i)
        chl_proj <- project(chl_r, "EPSG:6932")
        
        chl_crop <- if (is.na(letters_only[1]) && bbox == FALSE) {
          mask(crop(chl_proj, area_vector), area_vector)
        } else {
          mask(crop(chl_proj, extent_obj), extent_obj)
        }
        plot(chl_crop, col = chl_palette, breaks = chl_breaks, axes = FALSE, box = FALSE,legend = FALSE,)
        title(main = paste0(format(dates[i], "%b %Y")), cex.main = 2*multiplier, line = -2*(multiplier))
        if (asd_show) {
            plot(area_vector, add = TRUE, lwd = 0.75 * multiplier, border = 'black')
            centroids <- st_centroid(st_geometry(selected_areas))
            text(st_coordinates(centroids), labels = selected_areas$GAR_Long_L,col = "black", cex = 1 * multiplier)
        }
    } 
    dev.off()
    
}

In [ ]:
# ==============================================================================
# 7. ADD LEGEND BASED ON HORIZONTAL CONFIGURATION ----
# ==============================================================================
if (legend_pos != "No legend") {
    zlim <- c(0.03, index_max)
    round_to_half <- function(x) round(x * 2) / 2
    possible_breaks <- seq(0.5, round_to_half(index_max) - 0.5, by = 0.5)
    breaks_legend <- c(zlim[1], possible_breaks, index_max)
    labels_legend <- format(breaks_legend, nsmall = 2)
  n <- 5  # size factor

  # Path to save the legend image
  legend_file <- "outputs/Legend_Fig9.png"

  # Create the PNG file for the legend
  if (legend_pos == "Horizontal") {
    png(filename = legend_file, width = 400 * n, height = 75 * n, bg = "transparent")
    par(mar = c(2, 5, 2, 2))  # margins for horizontal legend
    horizontal = TRUE
  } else {
    png(filename = legend_file, width = 75 * n, height = 400 * n, bg = "transparent")
    par(mar = c(5, 2, 2, 5))  # margins for vertical legend
    horizontal = FALSE
  }

  # Draw the legend
  image.plot(
    zlim = zlim,
    col = chl_palette,
    legend.only = TRUE,
    horizontal = horizontal,
    legend.width = 2 * n,
    legend.shrink = 0.9,
    legend.mar = 8,
    axis.args = list(
      at = breaks_legend,
      labels = labels_legend,
      cex.axis = 2,
      col.axis = "black"
    )
  )
  dev.off()
}